In [ ]:
%pip install unbabel-comet
%pip install langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.0/91.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 825.4/825.4 kB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 102.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 84.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.9/438.9 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.1 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.65
    Uninstalling langchain-core-0.3.65:
      Successfully uninstalled langchain-core-0.3.65
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.25
    Uninstalling langchain-0.3.25:
      Successfully uninstalled langchain-0.3.25


# 🧪 Multilingual Translation Evaluation Framework with ChatGPT & COMET

This notebook evaluates English-to-multilingual translations using different prompting strategies with GPT-4o. It assesses both fluency (via COMET) and factual accuracy (via named entity preservation).

---

## 🔍 Evaluation Metrics

### 1. COMET Score (Semantic Quality)
COMET (Crosslingual Optimized Metric for Evaluation of Translation) is a neural metric that compares model-generated translations with human references.  
- Model used: `Unbabel/wmt22-comet-da`
- Scores range from 0 to 1 (higher is better)
- Captures fluency and semantic correctness

### 2. Meta Score (Entity Preservation)
A custom metric that checks if named entities (like people, places, products) are preserved correctly.  
- Extracts entity mentions from references
- Compares them against predictions
- Reports accuracy as a percentage

---

## 💬 Prompting Techniques

### Zero-Shot (Generic)
Basic instruction: “Translate the following English sentences into {target_language}.”  
No special guidance or examples.

### Zero-Shot (Entity-Aware)
Adds instructions to carefully translate named entities consistently and accurately.  
Improves factual precision.

### Few-Shot
Includes a few example translations (source + target) in the prompt to guide the model.  
Helps the model mimic correct patterns and handle low-resource scenarios.

### Chain-of-Thought (CoT)
Instructs the model to internally perform reasoning steps:  
1. Identify named entities  
2. Classify their types  
3. Translate entities contextually  
4. Translate the rest  
5. Combine everything  
Returns only final output in clean JSON format. Boosts consistency and control.

---

## ⚙️ Pipeline Summary

1. **Load input `.jsonl` files** from validation set
2. **Choose a prompt type** (zero-shot, few-shot, CoT)
3. **Translate in batches** (50 per request) with retry & rate limiting
4. **Save predictions** in structured JSON
5. **Evaluate** using:
   - `calculate_comet_scores()` for semantic fluency
   - `calculate_meta_score()` for entity translation accuracy
6. **Store results** per file in a `scores/` folder

---

## 📊 Example Evaluation Output

```json
{
  "correct_instances": 45,
  "total_instances": 50,
  "comet_score": 0.872,
  "meta_score": 90.0
}


In [ ]:
# ✅ ChatGPT-Compatible Framework for Translation Evaluation

import os
import re
import json
from typing import Dict, List, Set
from comet import download_model, load_from_checkpoint
from langchain_core.prompts import ChatPromptTemplate

# === Paths and Configuration ===
# Define training and validation data paths along with COMET model configuration
TRAIN_DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/semeval.train.v2-e0d1c28b78c8dd4969d25eea5d3bc9cc/semeval/train'
VALIDATION_DATA_DIR = '/content/drive/MyDrive/Colab Notebooks/semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation'
COMET_MODEL_NAME = "Unbabel/wmt22-comet-da"
COMET_NUM_GPUS = 1
COMET_BATCH_SIZE = 32

# Entity types to be evaluated in entity-level accuracy
ENTITY_TYPES = [
    "Musical work", "Artwork", "Food", "Animal", "Plant", "Book", "Book series",
    "Fictional entity", "Landmark", "Movie", "Place of worship", "Natural place",
    "TV series", "Person"
]

# === Utilities ===
# Convert language code to full name
def get_language_name(short_code):
    lang_map = {
        'ar': 'Arabic', 'zh': 'Chinese (Traditional)', 'fr': 'French', 'de': 'German',
        'it': 'Italian', 'ja': 'Japanese', 'ko': 'Korean', 'es': 'Spanish',
        'th': 'Thai', 'tr': 'Turkish', 'en': 'English'
    }
    return lang_map.get(short_code, short_code)

# Compute COMET scores between predictions and references
def download_comet_model():
    path = download_model(COMET_MODEL_NAME)
    return load_from_checkpoint(path)

def calculate_comet_scores(model, references_path, predictions_path):
    refs = _load_jsonl_data(references_path)
    preds = _load_jsonl_data(predictions_path)
    ids = set(refs.keys()) & set(preds.keys())

    instances, idx_map, idx = [], {}, 0
    for i in sorted(ids):
        for t in refs[i]['targets']:
            instances.append({
                "src": refs[i]['source'],
                "ref": t['translation'],
                "mt": preds[i]['prediction']
            })
        idx_map[i] = [idx, idx + len(refs[i]['targets'])]
        idx += len(refs[i]['targets'])

    print(f"Created {len(instances)} instances")
    scores = model.predict(instances, batch_size=COMET_BATCH_SIZE, gpus=COMET_NUM_GPUS).scores

    # Compute max score for each sample and average across all
    max_scores = [max(scores[start:end]) for start, end in idx_map.values()]
    avg_score = sum(max_scores) / (len(max_scores) + len(preds) - len(ids))
    print(f"Average COMET score: {100 * avg_score:.2f}")
    return avg_score

# Evaluate entity-level name translation accuracy
def calculate_meta_score(ref_path, pred_path, verbose=False):
    refs = _load_references(ref_path, ENTITY_TYPES)
    mentions = _get_mentions_from_references(refs)
    preds = _load_predictions(pred_path)
    acc = _compute_entity_name_translation_accuracy(preds, mentions, verbose)
    return acc['correct'], acc['total'], acc['accuracy'] * 100

# === Internal Helpers ===

# Load JSONL data with `id` as key
def _load_jsonl_data(path):
    return {json.loads(l)['id']: json.loads(l) for l in open(path, encoding='utf-8') if l.strip()}

# Load references with filtering based on entity type
def _load_references(path: str, types: List[str]) -> List[dict]:
    data = []
    for line in open(path, encoding='utf-8'):
        if not line.strip(): continue
        record = json.loads(line)
        if not record['targets']: continue
        if types and not any(e in record['entity_types'] for e in types): continue
        data.append(record)
    return data

def _load_predictions(path: str) -> Dict[str, str]:
    data = {}
    for line in open(path, encoding='utf-8'):
        if not line.strip(): continue
        record = json.loads(line)
        match = re.match(r"Q[0-9]+_[0-9]", record['id'])
        if not match: raise ValueError(f"Bad ID: {record['id']}")
        data[match.group(0)] = record['prediction']
    return data

# Load model predictions with validation on ID format
def _compute_entity_name_translation_accuracy(preds: Dict[str, str], mentions: Dict[str, Set[str]], verbose=False) -> dict:
    correct = sum(
        1 for k, m in mentions.items()
        if k in preds and any(v.casefold() in preds[k].casefold() for v in m)
    )
    total = len(mentions)
    return {"correct": correct, "total": total, "accuracy": correct / total if total else 0.0}

# Extract entity mentions from reference data
def _get_mentions_from_references(data: List[dict]) -> Dict[str, Set[str]]:
    return {d['id']: set(t['mention'] for t in d['targets']) for d in data}


In [ ]:
import json
import glob
import tqdm
import os
import time
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatOpenAI  # ✅ ChatGPT model
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential,
    retry_if_exception_type
)


# Set your OpenAI API key (ensure it's stored securely in production)
os.environ["OPENAI_API_KEY"] = ""
model_name="gpt-4o"
# Instantiate the ChatGPT model (can switch between gpt-3.5-turbo, gpt-4, gpt-4o)
llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


/tmp/ipython-input-2-1901089282.py:20: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-4o", temperature=0)


In [ ]:
# Basic zero-shot translation prompt
ZERO_SHOT_PROMPT_TEMPLATE_STRING_1 = """
Your task is to translate the following English sentences into {target_language}.
Input sentences are provided below as a JSON array of objects, each with an "id" and a "text" field.
Provide the translations as a JSON array of objects, where each object contains the original "id" and its "translation".
Maintain the original order of sentences from the input. Please return the json array without any additional text or formatting.

Input JSON:
{source_texts_json}
"""

# Named-entity-aware zero-shot prompt
ZERO_SHOT_PROMPT_TEMPLATE_STRING_2 = """
Your task is to translate the following English sentences into {target_language}.
Input sentences are provided below as a JSON array of objects, each with an "id" and a "text" field.
Ensure that **all named entities** (e.g., people, organizations, locations, product names) are translated **correctly and consistently** into the target language.
Provide the translations as a JSON array of objects, where each object contains the original "id" and its "translation".
Maintain the original order of sentences from the input. Please return the json array without any additional text or formatting.

Input JSON:
{source_texts_json}
"""

In [ ]:
# Rate limit for API calls (OpenAI recommends <20 RPM for gpt-4/gpt-4o)
REQUESTS_PER_MINUTE_LIMIT = 15

# Delay to enforce between requests (in seconds)
DELAY_BETWEEN_REQUESTS_SECONDS = 60 / REQUESTS_PER_MINUTE_LIMIT  # 4.0 sec

# Number of input samples to send per prompt
BATCH_SIZE = 50


In [ ]:
# Counter for API requests
api_request_count = 96

Phase 5: Translation Execution Logic

 What This Does:
Sends a batch of source texts to the LLM with retry support

Parses and verifies the returned JSON translations

Handles malformed output with clear fallback errors

Ensures reliable translation under API limits and occasional model inconsistencies.

In [ ]:
@retry(
    wait=wait_random_exponential(multiplier=1, min=DELAY_BETWEEN_REQUESTS_SECONDS, max=60),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(Exception)
)
def get_translated_content_with_retries(source_records_batch, target_language, llm_instance, template):
    global api_request_count

    # Prepare JSON input for the prompt
    input_json_for_prompt = [{"id": rec['id'], "text": rec['source']} for rec in source_records_batch]
    source_texts_json_str = json.dumps(input_json_for_prompt, ensure_ascii=False)

    # Format the prompt using the selected template
    prompt_to_send = template.format(
        source_texts_json=source_texts_json_str,
        target_language=target_language
    )

    # Invoke LLM with the prompt
    response = llm_instance.invoke(prompt_to_send)
    api_request_count += 1

    try:
        # Clean potential ```json markdown wrapper
        cleaned_content = response.content.strip()
        if cleaned_content.startswith("```json") and cleaned_content.endswith("```"):
            cleaned_content = cleaned_content[7:-3].strip()

        # Parse model output
        translated_outputs = json.loads(cleaned_content)

        # Validate expected structure
        if not isinstance(translated_outputs, list) or \
           not all(isinstance(item, dict) and 'id' in item and 'translation' in item for item in translated_outputs):
            raise ValueError("Model did not return a valid JSON array of translation objects.")

        # Map translations back to original IDs
        translated_dict = {item['id']: item['translation'] for item in translated_outputs}
        ordered_translations = []
        for record in source_records_batch:
            ordered_translations.append(translated_dict.get(record['id'], "ERROR: ID not found in JSON output"))

        return ordered_translations

    # Handle different error scenarios with fallback error messages
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Invalid JSON response from model"] * len(source_records_batch)
    except ValueError as e:
        print(f"Value Error (JSON format issue): {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Invalid JSON structure from model"] * len(source_records_batch)
    except Exception as e:
        print(f"An unexpected error occurred while processing model output: {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Unexpected issue processing model output"] * len(source_records_batch)


In [ ]:
# Folder containing input `.jsonl` files for validation
input_data_folder = "/content/drive/MyDrive/Colab Notebooks//semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation"

# List all .jsonl files in the folder
jsonl_files = glob.glob(f"{input_data_folder}/*.jsonl")

# Create an output folder for saving model predictions
output_prediction_dir = os.path.join("/content/drive/MyDrive/Colab Notebooks", "data/predictions", model_name, "validation")
os.makedirs(output_prediction_dir, exist_ok=True)


In [ ]:
jsonl_files = glob.glob(f"{input_data_folder}/*.jsonl")
jsonl_files

['/content/drive/MyDrive/Colab Notebooks//semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation/de_DE.jsonl',
 '/content/drive/MyDrive/Colab Notebooks//semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation/fr_FR.jsonl',
 '/content/drive/MyDrive/Colab Notebooks//semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation/th_TH.jsonl',
 '/content/drive/MyDrive/Colab Notebooks//semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation/ko_KR.jsonl',
 '/content/drive/MyDrive/Colab Notebooks//semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation/es_ES.jsonl',
 '/content/drive/MyDrive/Colab Notebooks//semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation/it_IT.jsonl',
 '/content/drive/MyDrive/Colab Notebooks//semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation/ja_JP.jsonl',
 '/content/drive/MyDrive/Colab Notebooks//semeval.validation.v2-889a1492ba6c3791baa8f4224bc8e685/validation/ar_AE.jsonl',
 '/content/drive/MyDrive

In [ ]:
def zero_shot_eval(template, template_id):
    overall_pbar = tqdm.tqdm(jsonl_files, desc="Processing files")

    # Create subfolder for this specific prompt variant
    output_prediction_dir_1 = os.path.join(output_prediction_dir, template_id)
    os.makedirs(output_prediction_dir_1, exist_ok=True)

    for file_path in overall_pbar:
        filename = os.path.basename(file_path)
        outfile_path = os.path.join(output_prediction_dir_1, filename)

        # Read input records from JSONL file
        data_to_translate = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data_to_translate.append(json.loads(line))

        translated_results_for_file = []

        # Process in batches
        for i in tqdm.tqdm(range(0, len(data_to_translate), BATCH_SIZE), desc=f"Translating {filename} in batches", leave=False):
            batch_records = data_to_translate[i : i + BATCH_SIZE]

            target_locale = batch_records[0]['target_locale']
            target_language = get_language_name(target_locale)

            try:
                # Run translation with retry support
                translated_texts_batch = get_translated_content_with_retries(
                    batch_records, target_language, llm, template
                )

                # Store results
                for j, record in enumerate(batch_records):
                    record_id = record['id']
                    source_text = record['source']
                    source_locale = record['source_locale']
                    translated_text = translated_texts_batch[j] if j < len(translated_texts_batch) else "ERROR: Translation missing"
                    translated_results_for_file.append({
                        "id": record_id,
                        "source_language": get_language_name(source_locale),
                        "target_language": target_language,
                        "text": source_text,
                        "prediction": translated_text,
                    })

            except Exception as e:
                print(f"\nCRITICAL ERROR: Failed to translate a batch starting with ID '{batch_records[0]['id']}' after multiple retries. Error: {e}")
                for record in batch_records:
                    translated_results_for_file.append({
                        "id": record['id'],
                        "source_language": get_language_name(record['source_locale']),
                        "target_language": get_language_name(record['target_locale']),
                        "text": record['source'],
                        "prediction": "ERROR: Batch translation failed due to API issues/rate limits.",
                    })

            # Respect API rate limits between batches/files
            if i + BATCH_SIZE < len(data_to_translate) or overall_pbar.n < len(jsonl_files):
                 time.sleep(DELAY_BETWEEN_REQUESTS_SECONDS)

        # Save translated output to file
        with open(outfile_path, 'w', encoding='utf-8') as f:
            for res in translated_results_for_file:
                f.write(json.dumps(res, ensure_ascii=False) + '\n')

        print(f"\nTranslations for {filename} saved to {outfile_path}")

    overall_pbar.close()
    print("All files processed and translations saved.")


In [ ]:
# Download and load the COMET model for evaluation
comet_model = download_comet_model()

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.48k [00:00<?, ?B/s]

hparams.yaml:   0%|          | 0.00/567 [00:00<?, ?B/s]

LICENSE:   0%|          | 0.00/9.69k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/3.40k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/616 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/pytorch_lightning/core/saving.py:195: Found keys that are not in the model state dict but in the checkpoint: ['encoder.model.embeddings.position_ids']


In [ ]:
def calculate_scores(template_id):
    # Create directory to save evaluation scores
    scores_dir = os.path.join(output_prediction_dir, template_id, "scores")
    if not os.path.exists(scores_dir):
        os.makedirs(scores_dir, exist_ok=True)

    for file_path in jsonl_files:
        references_path = file_path
        filename = os.path.basename(file_path)
        predictions_path = os.path.join(output_prediction_dir, template_id, filename)

        # Compute COMET score
        comet_score = calculate_comet_scores(
            comet_model,
            references_path,
            predictions_path
        )

        # Compute entity-level accuracy
        correct_instances, total_instances, meta_score = calculate_meta_score(
            references_path,
            predictions_path)

        # Package results into JSON
        evaluation_results = {
            "correct_instances": correct_instances,
            "total_instances": total_instances,
            "comet_score": comet_score,
            "meta_score": meta_score
        }

        # Save scores to file
        evaluation_output_path = os.path.join(scores_dir, f"{os.path.splitext(filename)[0]}.json")
        with open(evaluation_output_path, 'w', encoding='utf-8') as json_file:
            json.dump(evaluation_results, json_file, ensure_ascii=False, indent=4)


In [ ]:
# Build the zero-shot prompt using template string 1 (basic translation)
zero_shot_prompt_template = PromptTemplate(
    input_variables=["source_texts_json", "target_language"],
    template=ZERO_SHOT_PROMPT_TEMPLATE_STRING_1,
)

# Run evaluation pipeline: translate and save predictions
zero_shot_eval(zero_shot_prompt_template, "zero-shot-1")

# Evaluate predictions using COMET and entity-level accuracy
calculate_scores("zero-shot-1")


Processing files:  10%|█         | 1/10 [05:35<50:21, 335.68s/it]


Translations for de_DE.jsonl saved to /content/drive/MyDrive/Colab Notebooks/data/predictions/gpt-4o/validation/zero-shot-1/de_DE.jsonl



Processing files:  20%|██        | 2/10 [11:39<46:59, 352.42s/it]


Translations for fr_FR.jsonl saved to /content/drive/MyDrive/Colab Notebooks/data/predictions/gpt-4o/validation/zero-shot-1/fr_FR.jsonl



Processing files:  30%|███       | 3/10 [19:07<46:10, 395.76s/it]

...

Average COMET score: 90.86
Created 1260 instances


Predicting DataLoader 0: 100%|██████████| 40/40 [13:04<00:00, 19.60s/it]
INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Average COMET score: 92.65
Created 1544 instances


Predicting DataLoader 0: 100%|██████████| 49/49 [15:53<00:00, 19.45s/it]


Average COMET score: 92.28


In [ ]:
# Build the zero-shot prompt using template string 2 (with entity preservation)
zero_shot_prompt_template = PromptTemplate(
    input_variables=["source_texts_json", "target_language"],
    template=ZERO_SHOT_PROMPT_TEMPLATE_STRING_2,
)

# Run second evaluation pipeline with entity-focused prompt
zero_shot_eval(zero_shot_prompt_template, "zero-shot-2")

# Evaluate second run using COMET and entity accuracy
calculate_scores("zero-shot-2")



Processing files:   0%|          | 0/10 [00:00<?, ?it/s]

Translating de_DE.jsonl in batches:   0%|          | 0/15 [00:00<?, ?it/s]

Translating de_DE.jsonl in batches:   7%|▋         | 1/15 [00:17<04:04, 17.46s/it]

Translating de_DE.jsonl in batches:  13%|█▎        | 2/15 [00:44<04:58, 22.98s/it]

Translating de_DE.jsonl in batches:  20%|██        | 3/15 [01:11<05:00, 25.02s/it]

Translating de_DE.jsonl in batches:  27%|██▋       | 4/15 [01:32<04:16, 23.33s/it]

Translating de_DE.jsonl in batches:  33%|███▎      | 5/15 [01:53<03:45, 22.52s/it]

Translating de_DE.jsonl in batches:  40%|████      | 6/15 [02:13<03:14, 21.58s/it]

Translating de_DE.jsonl in batches:  47%|████▋     | 7/15 [02:32<02:47, 20.95s/it]

Translating de_DE.jsonl in batches:  53%|█████▎    | 8/15 [02:53<02:25, 20.84s/it]

Translating de_DE.jsonl in batches:  60%|██████    | 9/15 [03:13<02:02, 20.42s/it]

Translating de_DE.jsonl in batches:  67%|██████▋   | 10/15 [03:33<01:42, 20.42s/it]

Translating de_DE.jsonl 


Translations for de_DE.jsonl saved to /content/drive/MyDrive/Colab Notebooks/data/predictions/gpt-4o/validation/zero-shot-2/de_DE.jsonl




Translating fr_FR.jsonl in batches:   0%|          | 0/15 [00:00<?, ?it/s]

Translating fr_FR.jsonl in batches:   7%|▋         | 1/15 [00:20<04:48, 20.61s/it]

Translating fr_FR.jsonl in batches:  13%|█▎        | 2/15 [00:41<04:33, 21.00s/it]

Translating fr_FR.jsonl in batches:  20%|██        | 3/15 [01:03<04:17, 21.48s/it]

Translating fr_FR.jsonl in batches:  27%|██▋       | 4/15 [01:24<03:53, 21.21s/it]

Translating fr_FR.jsonl in batches:  33%|███▎      | 5/15 [01:44<03:26, 20.62s/it]

Translating fr_FR.jsonl in batches:  40%|████      | 6/15 [02:03<03:00, 20.04s/it]

Translating fr_FR.jsonl in batches:  47%|████▋     | 7/15 [02:28<02:55, 21.89s/it]

Translating fr_FR.jsonl in batches:  53%|█████▎    | 8/15 [02:53<02:38, 22.61s/it]

Translating fr_FR.jsonl in batches:  60%|██████    | 9/15 [03:13<02:11, 21.90s/it]

Translating fr_FR.jsonl in batches:  67%|██████▋   | 10/15 [03:33<01:46, 21.25s/it]

Translating fr_FR.jsonl in batches:  73%|███████▎  | 11/15 [03:54<01:24, 21.16s/i


Translations for fr_FR.jsonl saved to /content/drive/MyDrive/Colab Notebooks/data/predictions/gpt-4o/validation/zero-shot-2/fr_FR.jsonl




Translating th_TH.jsonl in batches:   0%|          | 0/15 [00:00<?, ?it/s]

Translating th_TH.jsonl in batches:   7%|▋         | 1/15 [00:27<06:27, 27.66s/it]

Translating th_TH.jsonl in batches:  13%|█▎        | 2/15 [00:52<05:37, 25.96s/it]

Translating th_TH.jsonl in batches:  20%|██        | 3/15 [01:18<05:11, 25.96s/it]

Translating th_TH.jsonl in batches:  27%|██▋       | 4/15 [01:44<04:47, 26.18s/it]

Translating th_TH.jsonl in batches:  33%|███▎      | 5/15 [02:12<04:27, 26.70s/it]

Translating th_TH.jsonl in batches:  40%|████      | 6/15 [02:36<03:50, 25.60s/it]

Translating th_TH.jsonl in batches:  47%|████▋     | 7/15 [03:01<03:25, 25.63s/it]

Translating th_TH.jsonl in batches:  53%|█████▎    | 8/15 [03:36<03:18, 28.42s/it]

Translating th_TH.jsonl in batches:  60%|██████    | 9/15 [04:02<02:45, 27.65s/it]

Translating th_TH.jsonl in batches:  67%|██████▋   | 10/15 [04:41<02:36, 31.36s/it]

Translating th_TH.jsonl in batches:  73%|███████▎  | 11/15 [05:05<01:56, 29.03s/i

...

Average COMET score: 88.55
Created 1260 instances


Predicting DataLoader 0: 100%|██████████| 40/40 [13:36<00:00, 20.42s/it]
INFO:pytorch_lightning.utilities.rank_zero:Using default `ModelCheckpoint`. Consider installing `litmodels` package to enable `LitModelCheckpoint` for automatic upload to the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Average COMET score: 92.57
Created 1544 instances


Predicting DataLoader 0: 100%|██████████| 49/49 [16:34<00:00, 20.29s/it]


Average COMET score: 92.11


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Few-shot prompt template for translation with example guidance
FEW_SHOT_PROMPT_TEMPLATE_STRING = """
Your task is to translate English sentences into {target_language}.
Below are a few examples of English sentences with named entities translated correctly into {target_language}.
Pay close attention to how named entities (e.g., people, organizations, locations, product names) are translated correctly and consistently.

Examples:
{few_shot_examples_json}

Now, translate the following new English sentences into {target_language}.
Provide the translations as a JSON array of objects, where each object contains the original "id" and its "translation" (ensure the key name is "translation").
Maintain the original order of sentences from the input. Please return the json array without any additional text or formatting.

Input JSON:
{source_texts_json}
"""


In [ ]:
few_shot_examples_dict = {
    'ar': [
        {'source': 'What is the seventh tallest mountain in North America?', 'target': 'ما سابع أعلى جبل في أمريكا الشمالية؟'},
        {'source': 'Which actor was the star of Titanic and was born in Los Angeles, California?', 'target': 'مَنْ الممثل الذي لعب دور البطولة في فيلم "تيتانيك" وهو من مواليد لوس أنجلوس بكاليفورنيا؟'},
        {'source': 'What year was the first book of the A Song of Ice and Fire series published?', 'target': 'في أي عام تم نشر أول كتاب من سلسلة "أغنية الجليد والنار"؟'},
        {'source': 'Who is the youngest current US governor?', 'target': 'مَن أصغر حاكم ولاية أمريكي حالٍ؟'},
        {'source': 'How long did it take to build the Lincoln Memorial?', 'target': 'كم من الوقت استغرق بناء نصب لينكولن التذكاري؟'},
        {'source': 'Has Bernie Sanders ever been president of the United States?', 'target': 'هل كان بيرني ساندرز رئيسًا للولايات المتحدة يومًا ما؟'},
        {'source': 'Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?', 'target': 'مَن الممثل الذي وقع عليه الاختيار الأول للكاتبة ستيفاني ماير للعب دور "إدوارد كولن" في فيلم "الشفق"؟'},
        {'source': 'Which river is longer than the Mississippi River?', 'target': 'أي نهر أطول من نهر المسيسيبي؟'},
        {'source': 'What is the latest US state to be admitted to the union that is not Hawaii?', 'target': 'ما آخر ولاية أمريكية تم قبولها في الاتحاد، بخلاف هاواي؟'},
        {'source': 'What is the longest lake in the world?', 'target': 'ما أطول بحيرة في العالم؟'},
    ],
    'de': [
        {'source': 'What is the seventh tallest mountain in North America?', 'target': 'Wie heißt der siebthöchste Berg Nordamerikas?'},
        {'source': 'What year was the first book of the A Song of Ice and Fire series published?', 'target': 'In welchem Jahr wurde das erste Buch der Reihe "Das Lied von Eis und Feuer" veröffentlicht?'},
        {'source': 'Who is the youngest current US governor?', 'target': 'Wer ist derzeit der jüngste amerikanische Gouverneur?'},
        {'source': 'Has Bernie Sanders ever been president of the United States?', 'target': 'War Bernie Sanders jemals Präsident der Vereinigten Staaten?'},
        {'source': 'Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?', 'target': 'Welcher Schauspieler war Stephanie Meyers erste Wahl für die Rolle des Edward Cullen in dem Film Twilight – Biss zum Morgengrauen?'},
        {'source': 'Which river is longer than the Mississippi River?', 'target': 'Welcher Fluss ist länger als der Mississippi'},
        {'source': 'What is the longest lake in the world?', 'target': 'Welcher See ist der längste der Welt?'},
        {'source': 'Is Texas the largest state in US?', 'target': 'Ist Texas der größte Bundesstaat in den Vereinigten Staaten?'},
        {'source': 'Who was the president of Argentina from 1989 to 1999?', 'target': 'Wer war von 1989 - 1999 Präsident von Argentinien?'},
        {'source': 'Who was a member of the 2004 U.S. Olympic mens swim team and born in Baltimore, Maryland?', 'target': 'Wer gehörte 2004 zur olympischen, Schwimmstaffel der Vereinigten Staaten und wurde in Baltimore, Maryland geboren?'},
    ],
    'es': [
        {'source': 'Which actor was the star of Titanic and was born in Los Angeles, California?', 'target': '¿Qué actor protagonizó Titanic y nació en Los Ángeles, California?'},
        {'source': 'What year was the first book of the A Song of Ice and Fire series published?', 'target': '¿En qué año se publicó el primer libro de la saga Canción de hielo y fuego?'},
        {'source': 'Which US president has had the most votes?', 'target': '¿Qué presidente de Estados Unidos obtuvo más votos?'},
        {'source': 'How long did it take to build the Lincoln Memorial?', 'target': '¿En cuánto tiempo se construyó el Monumento a Lincoln?'},
        {'source': 'Has Bernie Sanders ever been president of the United States?', 'target': '¿Bernie Sanders ha sido alguna vez presidente de los Estados Unidos?'},
        {'source': 'Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?', 'target': '¿Qué actor fue la primera opción de Stephanie Meyer para interpretar a Edward Cullen en la película Crepúsculo?'},
        {'source': 'What is the latest US state to be admitted to the union that is not Hawaii?', 'target': '¿Cuál es el último estado de Estados Unidos en ser incorporado a la unión aparte de Hawái?'},
        {'source': 'What is the longest lake in the world?', 'target': '¿Cuál es el lago más largo del mundo?'},
        {'source': 'Is Texas the largest state in US?', 'target': '¿Tejas es el estado más grande de Estados Unidos?'},
        {'source': 'How many times have the Los Angeles Dodgers lost the World Series?', 'target': '¿Cuántas veces perdieron los Dodgers de Los Ángeles la Serie Mundial?'},
    ],
    'fr': [
        {'source': 'What is the seventh tallest mountain in North America?', 'target': 'Quelle est la septième plus haute montagne d’Amérique du Nord ?'},
        {'source': 'Who is the youngest current US governor?', 'target': 'Qui est l’actuel plus jeune gouverneur américain ?'},
        {'source': 'Has Bernie Sanders ever been president of the United States?', 'target': 'Bernie Sanders a-t-il déjà été Président des États-Unis ?'},
        {'source': 'Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?', 'target': 'Quel acteur Stephanie Meyer a-t-elle choisi en premier pour jouer le rôle d’Edward Cullen dans le film Twilight ?'},
        {'source': 'Which river is longer than the Mississippi River?', 'target': 'Quel fleuve est plus long que le Mississippi ?'},
        {'source': 'What is the latest US state to be admitted to the union that is not Hawaii?', 'target': 'Quel est le dernier État Américain à avoir été admis dans l’Union et qui n’est pas Hawaï ?'},
        {'source': 'What is the longest lake in the world?', 'target': 'Quel est le lac le plus long du monde ?'},
        {'source': 'How many times have the Los Angeles Dodgers lost the World Series?', 'target': 'Combien de fois les Dodgers de Los Angeles ont-ils perdu dans la série mondiale ?'},
        {'source': 'Who was the president of Argentina from 1989 to 1999?', 'target': 'Qui était le Président de l’Argentine, de mille neuf cent quatre-vingt-neuf à mille neuf cent quatre-vingt-dix-neuf ?'},
        {'source': 'Who was a member of the 2004 U.S. Olympic mens swim team and born in Baltimore, Maryland?', 'target': 'Quel membre de l’équipe olympique masculine de natation des États-Unis de 2004 est né à Baltimore, dans le Maryland ?'},
    ],
    'it': [
        {'source': 'What year was the first book of the A Song of Ice and Fire series published?', 'target': 'In che anno venne pubblicato il primo libro della serie Cronache del Ghiaccio e del Fuoco?'},
        {'source': 'Which actor was Stephenie Meyers first choice to play Edward Cullen in the movie Twilight?', 'target': 'Quale attore era stata la prima scelta di Stephanie Meyer per interpretare Edward Cullen nel film Twilight?'},
        {'source': 'Which river is longer than the Mississippi River?', 'target': 'Quale fiume è più lungo del Mississippi?'},
        {'source': 'What is the longest lake in the world?', 'target': 'Qual è il lago più lungo del mondo?'},
        {'source': 'How many times have the Los Angeles Dodgers lost the World Series?', 'target': 'Quante volte i Los Angeles Dodgers hanno perso la Worls Series?'},
        {'source': 'Who was a member of the 2004 U.S. Olympic mens swim team and born in Baltimore, Maryland?', 'target': 'Chi era un membro della squadra olimpica di nuoto maschile americana del 2004 ed è nato a Baltimora, Maryland?'},
        {'source': 'Has there ever been a Christian U.S. senator?', 'target': 'Cè mai stato un senatore americano cristiano?'},
        {'source': 'How many Pulitzer Prizes has Colson Whitehead won?', 'target': 'Quanti premi Pulitzer ha vinto Colson Whitehead?'},
        {'source': 'Which movie was released first, Avengers: Endgame or Joker?', 'target': 'Quale film è uscito prima, Avengers Endgame o Joker?'},
        {'source': 'Was Friday the 13th released the same year as Halloween?', 'target': 'Venerdì 13 è uscito lo stesso anno di Halloween?'},
    ]
}

In [ ]:
from tenacity import retry, stop_after_attempt, wait_random_exponential, retry_if_exception_type

@retry(
    wait=wait_random_exponential(multiplier=1, min=DELAY_BETWEEN_REQUESTS_SECONDS, max=60),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(Exception)
)
def get_translated_content_with_retries_few_shot(source_records_batch, target_language, llm_instance, template, few_shot_examples):
    """
    Translates a batch of source_texts using the provided LLM instance and few-shot examples.
    Includes robust retry logic with exponential backoff for API stability.
    """
    global api_request_count

    # Prepare the input and examples JSON strings for templating
    input_json_for_prompt = [{"id": rec['id'], "text": rec['source']} for rec in source_records_batch]
    source_texts_json_str = json.dumps(input_json_for_prompt, ensure_ascii=False)
    few_shot_examples_json = json.dumps(few_shot_examples, ensure_ascii=False)

    # Format the few-shot prompt
    prompt_to_send = template.format(
        source_texts_json=source_texts_json_str,
        target_language=target_language,
        few_shot_examples_json=few_shot_examples_json
    )

    print(f"Sending prompt to model: {prompt_to_send}")  # Debug: Log full prompt

    # Invoke LLM with the prompt
    response = llm_instance.invoke(prompt_to_send)
    api_request_count += 1

    try:
        # Clean potential markdown wrapping
        cleaned_content = response.content.strip()
        if cleaned_content.startswith("```json") and cleaned_content.endswith("```"):
            cleaned_content = cleaned_content[7:-3].strip()

        # Parse model output
        translated_outputs = json.loads(cleaned_content)

        print(f"Model response: {translated_outputs}")  # Debug: Log parsed output

        # Validate structure
        if not isinstance(translated_outputs, list) or \
           not all(isinstance(item, dict) and 'id' in item and 'translation' in item for item in translated_outputs):
            raise ValueError("Model did not return a valid JSON array of translation objects.")

        # Reconstruct ordered translations
        translated_dict = {item['id']: item['translation'] for item in translated_outputs}
        ordered_translations = [translated_dict.get(record['id'], "ERROR: ID not found in JSON output")
                                for record in source_records_batch]

        return ordered_translations

    # Handle errors
    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Invalid JSON response from model"] * len(source_records_batch)
    except ValueError as e:
        print(f"Value Error (JSON format issue): {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Invalid JSON structure from model"] * len(source_records_batch)
    except Exception as e:
        print(f"An unexpected error occurred while processing model output: {e}. Raw response: {response.content.strip()}")
        return ["ERROR: Unexpected issue processing model output"] * len(source_records_batch)


In [ ]:
def few_shot_eval(template, template_id, few_shot_examples):
    overall_pbar = tqdm.tqdm(jsonl_files, desc="Processing files (few-shot)")

    # Create directory for saving few-shot predictions
    output_prediction_dir_1 = os.path.join(output_prediction_dir, template_id)
    os.makedirs(output_prediction_dir_1, exist_ok=True)

    for file_path in overall_pbar:
        filename = os.path.basename(file_path)
        outfile_path = os.path.join(output_prediction_dir_1, filename)

        # Load JSONL input file
        data_to_translate = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data_to_translate.append(json.loads(line))

        translated_results_for_file = []

        # Translate in batches
        for i in tqdm.tqdm(range(0, len(data_to_translate), BATCH_SIZE), desc=f"Translating {filename} in batches (few-shot)", leave=False):
            batch_records = data_to_translate[i : i + BATCH_SIZE]
            target_locale = batch_records[0]['target_locale']
            target_language = get_language_name(target_locale)

            try:
                # Translate batch using few-shot prompt
                translated_texts_batch = get_translated_content_with_retries_few_shot(
                    batch_records, target_language, llm, template, few_shot_examples[target_locale]
                )

                # Store translations with metadata
                for j, record in enumerate(batch_records):
                    record_id = record['id']
                    source_text = record['source']
                    source_locale = record['source_locale']
                    translated_text = translated_texts_batch[j] if j < len(translated_texts_batch) else "ERROR: Translation missing"
                    translated_results_for_file.append({
                        "id": record_id,
                        "source_language": get_language_name(source_locale),
                        "target_language": target_language,
                        "text": source_text,
                        "prediction": translated_text,
                    })

            except Exception as e:
                print(f"\nCRITICAL ERROR: Failed to translate a batch starting with ID '{batch_records[0]['id']}' after multiple retries. Error: {e}")
                for record in batch_records:
                    translated_results_for_file.append({
                        "id": record['id'],
                        "source_language": get_language_name(record['source_locale']),
                        "target_language": get_language_name(record['target_locale']),
                        "text": record['source'],
                        "prediction": "ERROR: Batch translation failed due to API issues/rate limits.",
                    })

            # Respect rate limits between batches/files
            if i + BATCH_SIZE < len(data_to_translate) or overall_pbar.n < len(jsonl_files):
                 time.sleep(DELAY_BETWEEN_REQUESTS_SECONDS)

        # Save results to file
        with open(outfile_path, 'w', encoding='utf-8') as f:
            for res in translated_results_for_file:
                f.write(json.dumps(res, ensure_ascii=False) + '\n')

        print(f"\nTranslations for {filename} saved to {outfile_path}")

    overall_pbar.close()
    print("All files processed and translations saved (few-shot).")


In [ ]:
# Build few-shot prompt template with support for in-context examples
few_shot_prompt_template = PromptTemplate(
    input_variables=["source_texts_json", "target_language", "few_shot_examples_json"],
    template=FEW_SHOT_PROMPT_TEMPLATE_STRING,
)

# Run the few-shot evaluation pipeline using provided examples
few_shot_eval(
    few_shot_prompt_template,
    "few-shot",
    few_shot_examples_dict
)


Translating de_DE.jsonl in batches (few-shot):   0%|          | 0/15 [00:00<?, ?it/s]

Sending prompt to model: 
Your task is to translate English sentences into German.
Below are a few examples of English sentences with named entities translated correctly into German.
Pay close attention to how named entities (e.g., people, organizations, locations, product names) are translated correctly and consistently.

Examples:
[{"source": "What is the seventh tallest mountain in North America?", "target": "Wie heißt der siebthöchste Berg Nordamerikas?"}, {"source": "What year was the first book of the A Song of Ice and Fire series published?", "target": "In welchem Jahr wurde das erste Buch der Reihe \"Das Lied von Eis und Feuer\" veröffentlicht?"}, {"source": "Who is the youngest current US governor?", "target": "Wer ist derzeit der jüngste amerikanische Gouverneur?"}, {"source": "Has Bernie Sanders ever been president of the United States?", "target": "War Bernie Sanders jemals Präsident der Vereinigten Staaten?"}, {"source": "Which actor was Stephenie Meyers first choice to pl


Translating de_DE.jsonl in batches (few-shot):   7%|▋         | 1/15 [00:28<06:41, 28.66s/it]

Sending prompt to model: 
Your task is to translate English sentences into German.
Below are a few examples of English sentences with named entities translated correctly into German.
Pay close attention to how named entities (e.g., people, organizations, locations, product names) are translated correctly and consistently.

Examples:
[{"source": "What is the seventh tallest mountain in North America?", "target": "Wie heißt der siebthöchste Berg Nordamerikas?"}, {"source": "What year was the first book of the A Song of Ice and Fire series published?", "target": "In welchem Jahr wurde das erste Buch der Reihe \"Das Lied von Eis und Feuer\" veröffentlicht?"}, {"source": "Who is the youngest current US governor?", "target": "Wer ist derzeit der jüngste amerikanische Gouverneur?"}, {"source": "Has Bernie Sanders ever been president of the United States?", "target": "War Bernie Sanders jemals Präsident der Vereinigten Staaten?"}, {"source": "Which actor was Stephenie Meyers first choice to pl


Translating de_DE.jsonl in batches (few-shot):  13%|█▎        | 2/15 [00:53<05:41, 26.29s/it]

...


Translating de_DE.jsonl in batches (few-shot):  20%|██        | 3/15 [01:15<04:53, 24.46s/it]

Sending prompt to model: 
Your task is to translate English sentences into German.
Below are a few examples of English sentences with named entities translated correctly into German.
Pay close attention to how named entities (e.g., people, organizations, locations, product names) are translated correctly and consistently.

Examples:
[{"source": "What is the seventh tallest mountain in North America?", "target": "Wie heißt der siebthöchste Berg Nordamerikas?"}, {"source": "What year was the first book of the A Song of Ice and Fire series published?", "target": "In welchem Jahr wurde das erste Buch der Reihe \"Das Lied von Eis und Feuer\" veröffentlicht?"}, {"source": "Who is the youngest current US governor?", "target": "Wer ist derzeit der jüngste amerikanische Gouverneur?"}, {"source": "Has Bernie Sanders ever been president of the United States?", "target": "War Bernie Sanders jemals Präsident der Vereinigten Staaten?"}, {"source": "Which actor was Stephenie Meyers first choice to pl


Processing files (few-shot):   0%|          | 0/10 [01:20<?, ?it/s]
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipython-input-16-2405482697.py", line 6, in <cell line: 0>
    few_shot_eval(
  File "/tmp/ipython-input-15-2179269205.py", line 25, in few_shot_eval
    translated_texts_batch = get_translated_content_with_retries_few_shot(
                             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tenacity/__init__.py", line 338, in wrapped_f
    return copy(f, *args, **kw)
           ^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tenacity/__init__.py", line 477, in __call__
    do = self.iter(retry_state=retry_state)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/tenacity/__init__.py", line 378, in iter
    result = action(retry_state)
             ^^^^^^^^^^^^^^^^^^

TypeError: object of type 'NoneType' has no len()

In [ ]:
calculate_scores("few-shot")

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Created 1260 instances


Predicting DataLoader 0: 100%|██████████| 40/40 [15:27<00:00, 23.19s/it]


Average COMET score: 89.88


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


...

Average COMET score: 33.04


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Created 1544 instances


Predicting DataLoader 0: 100%|██████████| 49/49 [16:09<00:00, 19.79s/it]


Average COMET score: 32.44


# Chain of Thought Prompting

In [ ]:
COT_PROMPT_TEMPLATE_STRING = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an expert linguist and translator. Your primary goal is to translate English sentences into {target_language}.
            You must pay special attention to identifying and accurately translating named entities within the sentence.

            Here's your step-by-step Chain-of-Thought process for each translation.
            Perform these steps internally, and then provide only the final JSON output as requested below.

            1.  **Identify Named Entities:**
                Go through each English sentence and meticulously identify all named entities.
                Named entities include, but are not limited to, persons, organizations, locations, dates, times, and specific titles.

            2.  **Determine Entity Type:**
                For each named entity identified in step 1, classify its specific type.
                Examples of types include: Person, Organization (ORG), Location (LOC), Date, Time, Title, Musical work, Artwork,
                Food, Animal, Plant, Book, Book series, Fictional entity, Landmark, Movie, Place of worship,
                Natural place, TV series.
                This classification provides crucial context for accurate translation.

            3.  **Contextual Translation of Named Entities:**
                For each identified named entity, determine its most appropriate translation into {target_language}.
                Consider the entity's type and the overall context of the sentence.
                - For proper nouns (like names of people or specific places), often a transliteration or the commonly accepted international form is best.
                - For organizations, use their official translated name if available, otherwise transliterate or provide a descriptive translation.
                - For dates and times, translate them according to the {target_language}'s conventions.

            4.  **Translate Remaining Sentence:**
                Translate all parts of the English sentence that are NOT named entities.
                Ensure that this translation is grammatically correct, natural-sounding, and culturally appropriate for {target_language}.

            5.  **Combine and Finalize Translation:**
                Integrate the accurately translated named entities (from step 3) back into the translated sentence (from step 4).
                Construct the final, complete, and fluent translation of the original English sentence.

            Your final output MUST be a JSON array of objects. Each object MUST contain two fields: "id" (the original sentence ID) and "translation" (the complete translated sentence).
            Do NOT include any other text, reasoning steps, or formatting outside of the JSON array.
            Example for JSON output:
            ```json
            [
              {{"id": "sentence_id_1", "translation": "Translated sentence 1."}},
              {{"id": "sentence_id_2", "translation": "Translated sentence 2."}}
            ]
            ```
            """
        ),
        ("human", "Translate the following English sentences into {target_language}:\n{source_texts_json}"),
    ]
)

In [ ]:
# Retry-enabled CoT translation handler
@retry(
    wait=wait_random_exponential(multiplier=1, min=DELAY_BETWEEN_REQUESTS_SECONDS, max=60),
    stop=stop_after_attempt(5),
    retry=retry_if_exception_type(Exception)
)
def get_translated_content_with_retries_using_cot(source_records_batch, target_language, llm_instance, template):
    global api_request_count

    # Prepare prompt input
    input_json_for_prompt = [{"id": rec['id'], "text": rec['source']} for rec in source_records_batch]
    source_texts_json_str = json.dumps(input_json_for_prompt, ensure_ascii=False)

    # Handle both ChatPromptTemplate and string template
    if isinstance(template, ChatPromptTemplate):
        response = llm_instance.invoke(template.format_messages(
            target_language=target_language,
            source_texts_json=source_texts_json_str
        ))
        raw_content = response.content
    else:
        prompt_to_send = template.format(
            source_texts_json=source_texts_json_str,
            target_language=target_language
        )
        response = llm_instance.invoke(prompt_to_send)
        raw_content = response.content

    api_request_count += 1

    try:
        # Clean markdown-wrapped JSON (```json ... ```)
        cleaned_content = raw_content.strip()
        if cleaned_content.startswith("```json") and cleaned_content.endswith("```"):
            cleaned_content = cleaned_content[7:-3].strip()

        translated_outputs = json.loads(cleaned_content)
        print(translated_outputs)  # Debug output

        # Validate format
        if not isinstance(translated_outputs, list) or \
           not all(isinstance(item, dict) and 'id' in item and 'translation' in item for item in translated_outputs):
            raise ValueError("Model did not return a valid JSON array of translation objects.")

        # Reorder results to match input
        translated_dict = {item['id']: item['translation'] for item in translated_outputs}
        ordered_translations = [translated_dict.get(record['id'], "ERROR: ID not found in JSON output")
                                for record in source_records_batch]

        return ordered_translations

    except json.JSONDecodeError as e:
        print(f"JSON Decode Error: {e}. Raw response: {raw_content.strip()}")
        return ["ERROR: Invalid JSON response from model"] * len(source_records_batch)
    except ValueError as e:
        print(f"Value Error (JSON format issue): {e}. Raw response: {raw_content.strip()}")
        return ["ERROR: Invalid JSON structure from model"] * len(source_records_batch)
    except Exception as e:
        print(f"An unexpected error occurred while processing model output: {e}. Raw response: {raw_content.strip()}")
        return ["ERROR: Unexpected issue processing model output"] * len(source_records_batch)

# Main evaluation loop for CoT or other flexible templates
def translation_eval_cot(template, template_id, few_shot_examples=None):
    overall_pbar = tqdm.tqdm(jsonl_files, desc=f"Processing files ({template_id})")

    output_prediction_sub_dir = os.path.join(output_prediction_dir, template_id)
    os.makedirs(output_prediction_sub_dir, exist_ok=True)

    for file_path in overall_pbar:
        filename = os.path.basename(file_path)
        outfile_path = os.path.join(output_prediction_sub_dir, filename)

        # Load input file
        data_to_translate = []
        with open(file_path, 'r', encoding='utf-8') as f:
            for line in f:
                data_to_translate.append(json.loads(line))

        translated_results_for_file = []

        for i in tqdm.tqdm(range(0, len(data_to_translate), BATCH_SIZE), desc=f"Translating {filename} in batches", leave=False):
            batch_records = data_to_translate[i : i + BATCH_SIZE]
            target_locale = batch_records[0]['target_locale']
            target_language = get_language_name(target_locale)

            # Handle dynamic few-shot partial or passthrough
            if isinstance(template, ChatPromptTemplate):
                current_template = template
            else:
                if few_shot_examples:
                    lang_specific_examples = [
                        ex for ex in few_shot_examples if get_language_name(ex['target_locale']) == target_language
                    ]
                    formatted_examples = json.dumps([
                        {"source": ex['source'], "target": ex['target']} for ex in lang_specific_examples
                    ], ensure_ascii=False)
                    current_template = template.partial(few_shot_examples_json=formatted_examples)
                else:
                    current_template = template

            try:
                # Run translation using selected template
                translated_texts_batch = get_translated_content_with_retries_using_cot(
                    batch_records, target_language, llm, current_template
                )

                # Format output
                for j, record in enumerate(batch_records):
                    record_id = record['id']
                    source_text = record['source']
                    source_locale = record['source_locale']
                    translated_text = translated_texts_batch[j] if j < len(translated_texts_batch) else "ERROR: Translation missing"
                    translated_results_for_file.append({
                        "id": record_id,
                        "source_language": get_language_name(source_locale),
                        "target_language": target_language,
                        "text": source_text,
                        "prediction": translated_text,
                    })

            except Exception as e:
                print(f"\nCRITICAL ERROR: Failed to translate a batch starting with ID '{batch_records[0]['id']}' after multiple retries. Error: {e}")
                for record in batch_records:
                    translated_results_for_file.append({
                        "id": record['id'],
                        "source_language": get_language_name(record['source_locale']),
                        "target_language": get_language_name(record['target_locale']),
                        "text": record['source'],
                        "prediction": "ERROR: Batch translation failed due to API issues/rate limits.",
                    })

            # Respect rate limits
            if i + BATCH_SIZE < len(data_to_translate) or overall_pbar.n < len(jsonl_files):
                time.sleep(DELAY_BETWEEN_REQUESTS_SECONDS)

        # Save predictions to file
        with open(outfile_path, 'w', encoding='utf-8') as f:
            for res in translated_results_for_file:
                f.write(json.dumps(res, ensure_ascii=False) + '\n')

        print(f"\nTranslations for {filename} saved to {outfile_path}")

    overall_pbar.close()
    print(f"All files processed for {template_id} and translations saved.")


In [ ]:
translation_eval_cot(COT_PROMPT_TEMPLATE_STRING, "cot")


In [ ]:
calculate_scores("cot")

INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.


Created 1260 instances


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
Predicting DataLoader 0: 100%|██████████| 40/40 [14:51<00:00, 22.28s/it]


Average COMET score: 89.89


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


...

Average COMET score: 92.24


INFO:pytorch_lightning.utilities.rank_zero:💡 Tip: For seamless cloud uploads and versioning, try installing [litmodels](https://pypi.org/project/litmodels/) to enable LitModelCheckpoint, which syncs automatically with the Lightning model registry.
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Created 1544 instances


Predicting DataLoader 0: 100%|██████████| 49/49 [16:36<00:00, 20.35s/it]


Average COMET score: 92.53


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.0 MB/s eta 0:00:00
